In [3]:
import pandas as pd

# --- Carga de los tres archivos CSV ---
try:
    df_articles = pd.read_csv('Datos/articles.csv')
    df_customers = pd.read_csv('Datos/customers.csv')
    df_trans = pd.read_csv('Datos/young_female_trans.csv') # Asegúrate que el nombre del archivo es correcto
except FileNotFoundError as e:
    print(f"Error: No se encontró el archivo {e.filename}. Asegúrate de que los archivos CSV están en el mismo directorio que tu script o notebook.")
    # Detenemos la ejecución si los archivos no se encuentran
    exit()

print("Archivos cargados exitosamente.")
print(f"Dimensiones - articles: {df_articles.shape}")
print(f"Dimensiones - customers: {df_customers.shape}")
print(f"Dimensiones - transacciones: {df_trans.shape}")

# --- Unión Estratégica de los DataFrames ---
# 1. Unir transacciones con la información de los clientes
print("\nUniendo transacciones con clientes...")
merged_df = pd.merge(df_trans, df_customers, on='customer_id', how='left')

# 2. Unir el resultado con la información de los artículos
print("Uniendo el resultado con los artículos...")
full_df = pd.merge(merged_df, df_articles, on='article_id', how='left')

print("\n¡Unión completada!")
print(f"Dimensiones del DataFrame final 'full_df': {full_df.shape}")
print("\nPrimeras 5 filas del DataFrame combinado 'full_df':")
print(full_df.head())

Archivos cargados exitosamente.
Dimensiones - articles: (105542, 25)
Dimensiones - customers: (1371980, 7)
Dimensiones - transacciones: (1337996, 5)

Uniendo transacciones con clientes...
Uniendo el resultado con los artículos...

¡Unión completada!
Dimensiones del DataFrame final 'full_df': (1337996, 35)

Primeras 5 filas del DataFrame combinado 'full_df':
        t_dat                                        customer_id  article_id  \
0  2018-09-20  00be0a263381af38132d31225e8fb12fbc527c654b4464...   644522001   
1  2018-09-20  012bbedf2efe728a7407a5dc842a852f8e09e9ae972711...   651456003   
2  2018-09-20  012bbedf2efe728a7407a5dc842a852f8e09e9ae972711...   651456003   
3  2018-09-20  012bbedf2efe728a7407a5dc842a852f8e09e9ae972711...   651456003   
4  2018-09-20  0137b87739a796f65396d8483173f66318039d19a2583f...   577992001   

      price  sales_channel_id   FN  Active club_member_status  \
0  0.059305                 2  1.0     1.0             ACTIVE   
1  0.016932                 1

In [4]:
categorical_cols = full_df.select_dtypes(include=['object', 'category']).columns.tolist()
print("Columnas categóricas detectadas automáticamente:", categorical_cols)


Columnas categóricas detectadas automáticamente: ['t_dat', 'customer_id', 'club_member_status', 'fashion_news_frequency', 'postal_code', 'prod_name', 'product_type_name', 'product_group_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name', 'index_code', 'index_name', 'index_group_name', 'section_name', 'garment_group_name', 'detail_desc']


In [5]:
print("\n--- Iniciando Paso 2: Limpieza y Preprocesamiento ---")

# --- Manejo de Valores Nulos ---
# Rellenar la edad faltante con la mediana
if 'age' in full_df.columns:
    median_age = full_df['age'].median()
    full_df['age'].fillna(median_age, inplace=True)
    print(f"Valores nulos en 'age' rellenados con la mediana: {median_age}")
else:
    print("Advertencia: La columna 'age' no se encontró en 'customers.csv'.")


# Rellenar nulos en columnas categóricas importantes
# Iteramos sobre una lista de columnas para mantener el código limpio
categorical_cols_to_fill = ['club_member_status', 'fashion_news_frequency']
for col in categorical_cols_to_fill:
    if col in full_df.columns:
        full_df[col].fillna('Unknown', inplace=True)
        print(f"Valores nulos en '{col}' rellenados con 'Unknown'.")

# --- Conversión de Tipos de Datos ---
# Convertir la columna de fecha a formato datetime
full_df['t_dat'] = pd.to_datetime(full_df['t_dat'])
print("Columna 't_dat' convertida a formato datetime.")

# Asegurarse de que los IDs son tratados como strings (categóricos)
full_df['customer_id'] = full_df['customer_id'].astype(str)
full_df['article_id'] = full_df['article_id'].astype(str)
print("Columnas de ID convertidas a tipo string.")

print("\n¡Limpieza y Preprocesamiento completados!")
print("\nInformación del DataFrame 'full_df' después de la limpieza:")
full_df.info()


--- Iniciando Paso 2: Limpieza y Preprocesamiento ---
Valores nulos en 'age' rellenados con la mediana: 25.0
Valores nulos en 'club_member_status' rellenados con 'Unknown'.
Valores nulos en 'fashion_news_frequency' rellenados con 'Unknown'.
Columna 't_dat' convertida a formato datetime.


C:\Users\crist\AppData\Local\Temp\ipykernel_25016\3919899993.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_df['age'].fillna(median_age, inplace=True)
C:\Users\crist\AppData\Local\Temp\ipykernel_25016\3919899993.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

Columnas de ID convertidas a tipo string.

¡Limpieza y Preprocesamiento completados!

Información del DataFrame 'full_df' después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337996 entries, 0 to 1337995
Data columns (total 35 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   t_dat                         1337996 non-null  datetime64[ns]
 1   customer_id                   1337996 non-null  object        
 2   article_id                    1337996 non-null  object        
 3   price                         1337996 non-null  float64       
 4   sales_channel_id              1337996 non-null  int64         
 5   FN                            1337996 non-null  float64       
 6   Active                        1337996 non-null  float64       
 7   club_member_status            1337996 non-null  object        
 8   fashion_news_frequency        1337996 non-null  object  

In [6]:
print("\n--- Iniciando Paso 3: Ingeniería de Características ---")

# --- 3.1. Creación de Perfiles de Usuario ---
print("Creando perfiles de usuario...")
user_profiles = full_df.groupby('customer_id').agg(
    # Característica demográfica
    age=('age', 'first'),  # 'first' toma el primer valor, ya que es constante por usuario
    
    # Características de comportamiento
    avg_price_paid=('price', 'mean'),
    total_articles_bought=('article_id', 'count'),
    
    # Características de "Gusto" (usando la moda)
    # Usamos una función lambda para manejar casos donde un usuario no tiene datos para una categoría
    fav_product_type=('product_type_name', lambda x: x.mode()[0] if not x.empty else 'Unknown'),
    fav_color=('colour_group_name', lambda x: x.mode()[0] if not x.empty else 'Unknown'),
    fav_department=('department_name', lambda x: x.mode()[0] if not x.empty else 'Unknown')
).reset_index()

print("Perfiles de usuario creados exitosamente.")
print(user_profiles.head())

# --- 3.2. Creación de Características de Producto (Refinado) ---
print("\nCreando características de producto (versión refinada)...")

# Calcular popularidad del artículo
item_popularity = full_df.groupby('article_id').agg(
    times_purchased=('customer_id', 'count')
).reset_index()

# --- SELECCIÓN EXPLÍCITA DE FEATURES ---
# Seleccionamos las características categóricas que son útiles y tienen una cardinalidad manejable.
# EXCLUIMOS: prod_name, detail_desc, y columnas de código/ID que no usaremos.
good_static_item_features = [
    'article_id', 'product_type_name', 'product_group_name', 'graphical_appearance_name', 
    'colour_group_name', 'department_name', 'index_name', 'section_name', 'garment_group_name'
]
# Asegurarnos de que todas las columnas seleccionadas existen en el DataFrame
existing_good_features = [col for col in good_static_item_features if col in df_articles.columns]

static_item_features = df_articles[existing_good_features].copy()
static_item_features['article_id'] = static_item_features['article_id'].astype(str)

# Unir características estáticas con las dinámicas (popularidad)
final_item_features = pd.merge(static_item_features, item_popularity, on='article_id', how='left')
final_item_features['times_purchased'].fillna(0, inplace=True)

print("Características de producto refinadas y creadas exitosamente.")
print(final_item_features.head())


--- Iniciando Paso 3: Ingeniería de Características ---
Creando perfiles de usuario...
Perfiles de usuario creados exitosamente.
                                         customer_id   age  avg_price_paid  \
0  00066fdcf5f0da690b898b287d05ce477bd2764ce975d1...  28.0        0.026779   
1  0010e8eb18f131e724d6997909af0808adbba057529edb...  25.0        0.026849   
2  0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...  27.0        0.024092   
3  00155b2ef48cfb5d2fce4642f670f151efe0747542a5b9...  21.0        0.026822   
4  001a7fb6def4cc4de27cb02f0025ea28c8ee74efdd3c73...  24.0        0.017949   

   total_articles_bought fav_product_type fav_color        fav_department  
0                     21           Shorts      Blue  Denim Other Garments  
1                     90         Trousers     Black          Jersey Basic  
2                     19         Trousers     Black              Swimwear  
3                     67          Sweater     Black       Casual Lingerie  
4                    

C:\Users\crist\AppData\Local\Temp\ipykernel_25016\678449916.py:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_item_features['times_purchased'].fillna(0, inplace=True)


In [8]:
import numpy as np

print("\n--- Iniciando Paso 4: Creación del Dataset de Entrenamiento ---")

# --- 4.1. Crear Ejemplos Positivos ---
positive_samples = full_df[['customer_id', 'article_id']].drop_duplicates().copy()
positive_samples['target'] = 1
print(f"Número de ejemplos positivos (interacciones únicas): {len(positive_samples)}")

# --- 4.2. Crear Ejemplos Negativos (Negative Sampling) ---
print("Generando ejemplos negativos... (Esto puede tardar un poco)")

# Crear un set de todos los IDs de artículos para una búsqueda eficiente
all_article_ids = set(df_articles['article_id'].astype(str))

# Diccionario para guardar los artículos comprados por cada usuario
user_purchased_items = positive_samples.groupby('customer_id')['article_id'].apply(set)

negative_samples_list = []
negative_sample_ratio = 2 # Generar 2 negativos por cada positivo

for customer_id, purchased_set in user_purchased_items.items():
    # Artículos que el usuario no ha comprado
    non_purchased_items = all_article_ids - purchased_set
    
    # Determinar cuántos negativos muestrear
    num_neg_samples = len(purchased_set) * negative_sample_ratio
    
    # Tomar una muestra aleatoria de los no comprados
    # Usamos min() para evitar errores si un usuario ha comprado casi todo
    samples_to_take = min(num_neg_samples, len(non_purchased_items))
    if samples_to_take > 0:
        negative_choices = np.random.choice(list(non_purchased_items), size=samples_to_take, replace=False)
        
        for article_id in negative_choices:
            negative_samples_list.append({'customer_id': customer_id, 'article_id': article_id, 'target': 0})

negative_samples = pd.DataFrame(negative_samples_list)
print(f"Número de ejemplos negativos generados: {len(negative_samples)}")

# --- 4.3. Unir todo para el Dataset Final ---
print("\nCombinando positivos, negativos y features...")

# 1. Concatenar positivos y negativos
training_data = pd.concat([positive_samples, negative_samples], ignore_index=True)

# 2. Añadir las características del usuario
training_data = pd.merge(training_data, user_profiles, on='customer_id', how='left')

# 3. Añadir las características del producto
training_data = pd.merge(training_data, final_item_features, on='article_id', how='left')

# 4. Limpieza final: Rellenar posibles nulos que hayan surgido del merge
training_data.fillna(0, inplace=True) # Una estrategia simple es rellenar con 0

# 5. Mezclar el dataset para eliminar cualquier orden
training_data = training_data.sample(frac=1, random_state=42).reset_index(drop=True)

print("\n¡Dataset de Entrenamiento 'training_data' creado y listo!")
print(f"Dimensiones finales: {training_data.shape}")
print("Distribución del target:")
print(training_data['target'].value_counts())
print("\nPrimeras filas del dataset de entrenamiento final:")
print(training_data.head())


# Guardar el DataFrame en formato Parquet
output_path = 'training_data.parquet'
training_data.to_parquet(output_path, engine='pyarrow')

print(f"¡Dataset de entrenamiento final guardado exitosamente en '{output_path}'!")


--- Iniciando Paso 4: Creación del Dataset de Entrenamiento ---
Número de ejemplos positivos (interacciones únicas): 1120412
Generando ejemplos negativos... (Esto puede tardar un poco)
Número de ejemplos negativos generados: 2240824

Combinando positivos, negativos y features...

¡Dataset de Entrenamiento 'training_data' creado y listo!
Dimensiones finales: (3361236, 18)
Distribución del target:
target
0    2240824
1    1120412
Name: count, dtype: int64

Primeras filas del dataset de entrenamiento final:
                                         customer_id article_id  target   age  \
0  bf97c1a3e662bdc94bc917fcd81fc0923dd16bce6c2ec7...  691110001       0  29.0   
1  78425dc9f364ee0fc7734f3204595f9ead322622bba183...  793111001       0  23.0   
2  6cd6688b2571bd55f6b5d074aa6129ad7e6751f027153c...  672383004       0  24.0   
3  39762eb23aa18643a655d99bc22b968cf3c4a4551bff1e...  748440001       0  22.0   
4  b8f3c05e5318b7c9b0983dc15d1a9cfddfce12ac674fe3...  762585001       1  28.0   

  